In [1]:
import osmnx as ox
import networkx as nx
import numpy as np
import random
from mesa import Model, Agent
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.time import SimultaneousActivation

# 茂木町のOSMネットワーク
G = ox.graph_from_place("茂木町, 栃木県, 日本", network_type="drive")

# 範囲・スケーリング設定
node_xs = [data['x'] for _, data in G.nodes(data=True)]
node_ys = [data['y'] for _, data in G.nodes(data=True)]
min_x, min_y = min(node_xs), min(node_ys)
scale = 100  # 約100倍スケール → 1グリッド≒10m

# OSM→Mesa座標変換関数
def osm_to_grid(x, y, min_x, min_y, scale=100):
    return int((x - min_x) * scale), int((y - min_y) * scale)

# 施設（緯度, 経度）→ノード→グリッド座標
facility_coords = {
    "hospital": (36.529847, 140.147155),
    "supermarket": (36.535582, 140.136053),
    "cityhall": (36.527175, 140.145986)
}
facility_nodes = {k: ox.distance.nearest_nodes(G, lon, lat) for k, (lat, lon) in facility_coords.items()}
facility_grids = {
    k: osm_to_grid(G.nodes[n]["x"], G.nodes[n]["y"], min_x, min_y, scale)
    for k, n in facility_nodes.items()
}

class ElderAgent(Agent):
    def __init__(self, unique_id, model, node_id, destination_label):
        super().__init__(unique_id, model)
        self.node_id = node_id
        self.destination_label = destination_label
        self.destination_node = facility_nodes[destination_label]
        self.wait_time = 0

        # グリッド上の位置に変換
        x, y = G.nodes[self.node_id]['x'], G.nodes[self.node_id]['y']
        grid_x, grid_y = osm_to_grid(x, y, min_x, min_y, scale)
        self.model.grid.place_agent(self, (grid_x, grid_y))

    def step(self):
        self.wait_time += 1

class CarAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.route = []
        self.index = 0

    def set_route_by_nodes(self, path_nodes):
        self.route = path_nodes
        self.index = 0

    def step(self):
        if self.index < len(self.route):
            node_id = self.route[self.index]
            x, y = G.nodes[node_id]["x"], G.nodes[node_id]["y"]
            grid_x, grid_y = osm_to_grid(x, y, min_x, min_y, scale)
            self.model.grid.move_agent(self, (grid_x, grid_y))
            self.index += 1


In [2]:
import osmnx as ox
import networkx as nx
import random
import pandas as pd
from mesa import Agent, Model
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.time import SimultaneousActivation

# --- 茂木町のネットワークを取得 ---
G = ox.graph_from_place("Motegi, Tochigi, Japan", network_type="drive")

# --- OSMノードの最小座標・スケール設定 ---
node_xs = [data['x'] for _, data in G.nodes(data=True)]
node_ys = [data['y'] for _, data in G.nodes(data=True)]
min_x, min_y = min(node_xs), min(node_ys)
scale = 100

# --- OSM → Mesa座標変換 ---
def osm_to_grid(x, y, min_x, min_y, scale=100):
    return int((x - min_x) * scale), int((y - min_y) * scale)

# --- 施設定義 ---
facility_coords = {
    "hospital": (36.529847, 140.147155),
    "supermarket": (36.535582, 140.136053),
    "cityhall": (36.527175, 140.145986)
}
facility_nodes = {k: ox.distance.nearest_nodes(G, lon, lat) for k, (lat, lon) in facility_coords.items()}
facility_grids = {
    k: osm_to_grid(G.nodes[n]["x"], G.nodes[n]["y"], min_x, min_y, scale)
    for k, n in facility_nodes.items()
}

class ElderAgent(Agent):
    def __init__(self, unique_id, model, node_id, destination_label):
        super().__init__(unique_id, model)
        self.node_id = node_id
        self.destination_label = destination_label
        self.destination_node = facility_nodes[destination_label]
        self.wait_time = 0
        self.transported = False

        # 座標変換して配置
        x, y = G.nodes[self.node_id]['x'], G.nodes[self.node_id]['y']
        grid_x, grid_y = osm_to_grid(x, y, min_x, min_y, scale)
        self.model.grid.place_agent(self, (grid_x, grid_y))

    def step(self):
        if not self.transported:
            self.wait_time += 1

class CarAgent(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.route = []
        self.index = 0
        self.total_distance = 0
        self.transports = []
        self.purpose_log = {"hospital": 0, "supermarket": 0, "cityhall": 0}

    def step(self):
        # ピックアップ対象がいなければ探索して運ぶ
        waiting_elders = [a for a in self.model.schedule.agents if isinstance(a, ElderAgent) and not a.transported]
        if waiting_elders:
            for elder in waiting_elders:
                try:
                    pickup = nx.shortest_path(G, self.model.current_node, elder.node_id, weight='length')
                    dropoff = nx.shortest_path(G, elder.node_id, elder.destination_node, weight='length')
                    full_route = pickup + dropoff[1:]

                    self.total_distance += sum(
                        ox.distance.euclidean_dist_vec(G.nodes[a]['y'], G.nodes[a]['x'], G.nodes[b]['y'], G.nodes[b]['x'])
                        for a, b in zip(full_route[:-1], full_route[1:])
                    )

                    self.model.current_node = full_route[-1]
                    elder.transported = True
                    self.purpose_log[elder.destination_label] += 1
                    self.transports.append(elder.wait_time)

                except:
                    pass  # 経路探索失敗時

    def get_average_wait(self):
        if self.transports:
            return sum(self.transports) / len(self.transports)
        return 0

class TransportModel(Model):
    def __init__(self, steps=10):
        super().__init__()
        self.schedule = SimultaneousActivation(self)
        self.grid = MultiGrid(200, 200, torus=False)
        self.steps = steps
        self.current_node = random.choice(list(G.nodes))
        self.car = CarAgent("car_1", self)
        self.schedule.add(self.car)
        self.datacollector = DataCollector(
            model_reporters={
                "Transports": lambda m: len(self.car.transports),
                "AvgWait": lambda m: self.car.get_average_wait(),
                "Distance": lambda m: self.car.total_distance,
                "Hospital": lambda m: self.car.purpose_log["hospital"],
                "Supermarket": lambda m: self.car.purpose_log["supermarket"],
                "Cityhall": lambda m: self.car.purpose_log["cityhall"]
            }
        )

    def step(self):
        # 高齢者を50%の確率で出現
        if random.random() < 0.5:
            elder_node = random.choice(list(G.nodes))
            purpose = random.choices(
                ["hospital", "supermarket", "cityhall"],
                weights=[0.4, 0.4, 0.2]
            )[0]
            elder = ElderAgent(f"elder_{self.schedule.time}", self, elder_node, purpose)
            self.schedule.add(elder)

        self.schedule.step()
        self.datacollector.collect(self)


In [3]:
model = TransportModel(steps=20)
for _ in range(20):
    model.step()

results = model.datacollector.get_model_vars_dataframe()
print(results)

# CSV保存
results.to_csv("transport_simulation_results.csv", index=False)



    Transports  AvgWait  Distance  Hospital  Supermarket  Cityhall
0            0        0         0         0            0         0
1            0        0         0         0            0         0
2            0        0         0         0            0         0
3            0        0         0         0            0         0
4            0        0         0         0            0         0
5            0        0         0         0            0         0
6            0        0         0         0            0         0
7            0        0         0         0            0         0
8            0        0         0         0            0         0
9            0        0         0         0            0         0
10           0        0         0         0            0         0
11           0        0         0         0            0         0
12           0        0         0         0            0         0
13           0        0         0         0            0      

In [14]:
import random
import networkx as nx
import osmnx as ox
import pandas as pd
from mesa import Agent, Model
from mesa.time import RandomActivation



# 施設ノードをランダムに設定（のちに固定でも可）
facility_nodes = {
    "hospital": random.choice(list(G.nodes)),
    "supermarket": random.choice(list(G.nodes)),
    "cityhall": random.choice(list(G.nodes))
}

# 高齢者エージェント
class ElderAgent(Agent):
    def __init__(self, unique_id, model, node_id, destination_node, destination_label):
        super().__init__(unique_id, model)
        self.node_id = node_id
        self.destination_node = destination_node
        self.destination_label = destination_label
        self.transported = False
        self.wait_time = 0

    def step(self):
        if not self.transported:
            self.wait_time += 1

# 車エージェント
class CarAgent(Agent):
    def __init__(self, unique_id, model, G, start_node=None):
        super().__init__(unique_id, model)
        self.G = G
        self.total_distance = 0
        self.transports = []
        self.purpose_log = {"hospital": 0, "supermarket": 0, "cityhall": 0}
        self.travel_log = []
        self.current_node = start_node or random.choice(list(G.nodes))

    def step(self):
        elders = [a for a in self.model.schedule.agents if isinstance(a, ElderAgent) and not a.transported]
        step_distance = 0

        for elder in elders:
            try:
                pickup_path = nx.shortest_path(self.G, self.current_node, elder.node_id, weight='length')
                dropoff_path = nx.shortest_path(self.G, elder.node_id, elder.destination_node, weight='length')
                full_path = pickup_path + dropoff_path[1:]

                # 移動距離（euclidean）計算
                dist = sum(
                    G.edges[a, b, 0].get("length", 0)
                    for a, b in zip(full_path[:-1], full_path[1:])
                )

                self.total_distance += dist
                step_distance += dist
                self.current_node = full_path[-1]
                self.travel_log.extend(full_path)

                elder.transported = True
                self.transports.append(elder.wait_time)
                self.purpose_log[elder.destination_label] += 1

            except nx.NetworkXNoPath:
                continue

        return step_distance


# モデル
class TransportModel(Model):
    def __init__(self, G,steps=20, elder_prob=0.5):
        super().__init__()
        self.G = G
        self.schedule = RandomActivation(self)
        self.steps = steps
        self.current_step = 0
        self.elder_prob = elder_prob
        self.step_stats = []

        # 車両エージェント2台
        for _ in range(2):
            car = CarAgent(self.next_id(), self,G)
            self.schedule.add(car)

    def step(self):
        step_distance = 0
        transported_this_step = 0
        wait_times = []

        # 高齢者エージェント生成（50%の確率）
        for _ in range(5):
            if random.random() < self.elder_prob:
                node_id = random.choice(list(G.nodes))
                destination_label = random.choices(["hospital", "supermarket", "cityhall"], weights=[0.4, 0.4, 0.2])[0]
                destination_node = facility_nodes[destination_label]
                elder = ElderAgent(self.next_id(), self, node_id, destination_node, destination_label)
                self.schedule.add(elder)

        self.schedule.step()

        for agent in self.schedule.agents:
            if isinstance(agent, CarAgent):
                d = agent.step()  # 🚗 ← 距離を返すようにしてある！
                step_distance += d
                transported_this_step += len(agent.transports)
                wait_times.extend(agent.transports)
                agent.transports = []

        avg_wait = sum(wait_times) / len(wait_times) if wait_times else 0

        hospital_count = sum(a.purpose_log["hospital"] for a in self.schedule.agents if isinstance(a, CarAgent))
        supermarket_count = sum(a.purpose_log["supermarket"] for a in self.schedule.agents if isinstance(a, CarAgent))
        cityhall_count = sum(a.purpose_log["cityhall"] for a in self.schedule.agents if isinstance(a, CarAgent))

        self.step_stats.append({
            "step": self.current_step,
            "transported": transported_this_step,
            "avg_wait": avg_wait,
            "distance": step_distance,
            "hospital": hospital_count,
            "supermarket": supermarket_count,
            "cityhall": cityhall_count
        })

        self.current_step += 1

# 実行と結果出力
# モデルを初期化・実行
G = ox.graph_from_place("茂木町, 栃木県, 日本", network_type="drive")
model = TransportModel(G=G, steps=20)
for _ in range(model.steps):
    model.step()


df = pd.DataFrame(model.step_stats)
df.to_csv("stepwise_stats.csv", index=False)
print(df)


    step  transported  avg_wait  distance  hospital  supermarket  cityhall
0      0            0  0.000000         0         0            0         0
1      1            2  0.500000         0         1            1         0
2      2            4  0.250000         0         3            2         1
3      3            1  0.000000         0         3            3         1
4      4            3  0.000000         0         4            5         1
5      5            1  0.000000         0         4            6         1
6      6            3  0.000000         0         4            8         2
7      7            2  0.000000         0         5            9         2
8      8            3  0.333333         0         5           11         3
9      9            2  1.000000         0         6           12         3
10    10            3  0.000000         0         7           13         4
11    11            1  1.000000         0         8           13         4
12    12            3  0.

In [12]:
m = folium.Map(location=[36.53, 140.14], zoom_start=14)

features = []
for e in model.elder_locations:
    features.append({
        "type": "Feature",
        "geometry": {"type": "Point", "coordinates": [e["lon"], e["lat"]]},
        "properties": {
            "time": f"2025-01-01T{e['time']:02d}:00:00",
            "style": {"color": "red"},
            "icon": "circle",
            "popup": f"Elder to {e['purpose']}"
        }
    })

geojson = {"type": "FeatureCollection", "features": features}

TimestampedGeoJson(
    geojson,
    period="PT1H",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="HH:mm",
    time_slider_drag_update=True
).add_to(m)

m


AttributeError: 'TransportModel' object has no attribute 'elder_locations'